In [55]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import csv
import lxml.html as lh

## Automated Course ID Retrieval

In [109]:
driver = webdriver.Chrome()

In [107]:
filename = "states_urls.csv"
  
# initializing the titles and rows list 
rows = [] 
  
# reading csv file 
with open(filename, 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = csvreader.__next__()
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row)

In [ ]:
city_url_list = []

for url in rows:
    driver.get(url[0])
    time.sleep(2)
    
    course_list = driver.find_element_by_class_name("LocationTagPage-sublocations-content").get_attribute('outerHTML')
    soup = BeautifulSoup(course_list,'html.parser')
    cities = soup.findAll(lambda tag: tag.name == 'div' and tag['class'] == ['LocationTagPromo-description'])
    course_list = []
    
    for city in cities:
        if '0 courses' in city.text:
            course_list.append(False)
        else:
            course_list.append(True)
        
    counter = 0
    
    for link in soup.findAll('a'):
        course_link = link.get('href')
        if(course_list[counter]==True):
            city_url_list.append(course_link)
        counter+= 1
    
    print(len(city_url_list))
    
    time.sleep(2)

In [118]:
city_df = pd.DataFrame(city_url_list)
city_df.to_csv('cities_url.csv',index=False)

In [114]:
with open('cities_url.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(city_url_list)

In [110]:
driver.get("https://www.golfadvisor.com/course-directory/2-usa/")

In [ ]:
course_list = driver.find_element_by_xpath("/html/body/div[2]/main/div[1]/div[5]/div/div/ul").get_attribute('outerHTML')
soup = BeautifulSoup(course_list,'html.parser')
for link in soup.findAll('a'):
    course_link = link.get('href')
    print(course_link)

In [46]:
course_name = driver.find_element_by_class_name("CoursePage-pageLeadHeading").text
address = driver.find_element_by_class_name("CoursePageSidebar-addressFirst").text
access = driver.find_element_by_xpath("/html/body/div[2]/div/main/div[1]/div[3]/div/div[1]/div[1]/div/div[2]/div/span[2]").text
year_built = driver.find_element_by_xpath("/html/body/div[2]/div/main/div[1]/div[3]/div/div[2]/div/div[1]/div[1]/div/span[2]").text
rating = driver.find_element_by_class_name("RatingStarItem-stars-value").text
num_reviews = driver.find_element_by_xpath("/html/body/div[2]/div/main/div[1]/div[2]/div/div[2]/div[1]/div/div/div[1]/div[3]/div/span[2]").text
pct_recommend = driver.find_element_by_class_name("RatingRecommendation-percentValue").text
architects = driver.find_element_by_xpath("/html/body/div[2]/div/main/div[1]/div[3]/div/div[2]/div/div[2]").text


''

In [ ]:
all_courses = pd.DataFrame()

for i in range(2,53):
    print(i)
    driver.find_element_by_xpath(f"/html/body/div[2]/div/form/div[4]/ol/li[4]/select/option[{i}]").click()
    time.sleep(5)
    driver.find_element_by_id("myButton").click()
    time.sleep(10)
    tbl = driver.find_element_by_xpath("/html/body/div[2]/div/form/div[10]/div[1]/table").get_attribute('outerHTML')
    course_details = pd.read_html(tbl)[0]
    course_id_list = []
    time.sleep(2)
    soup = BeautifulSoup(tbl,'html.parser')
    for link in soup.findAll('a'):
        course_id = link.get('href')
        course_id = course_id.split("=",1)[1]
        course_id_list.append(course_id)
    course_id_df = pd.DataFrame(course_id_list)
    course_id_df = course_id_df.rename(columns={0: "Course ID"})
    course_details['Course ID'] = course_id_df
    all_courses = all_courses.append(course_details).reset_index(drop=True)

all_courses.head()

In [ ]:
all_courses.to_csv('course_id_directory.csv',index=False)

## Retrieve Course IDs one at a time

In [ ]:
driver.get("https://ncrdb.usga.org/NCRDB/")

In [ ]:
print(i)
driver.find_element_by_xpath(f"/html/body/div[2]/div/form/div[4]/ol/li[4]/select/option[{i}]").click()

In [ ]:
driver.find_element_by_xpath("/html/body/div[2]/div/form/div[5]/input").click()

### Wait for page to load...

In [ ]:
tbl = driver.find_element_by_xpath("/html/body/div[2]/div/form/div[10]/div[1]/table").get_attribute('outerHTML')

In [ ]:
course_details = pd.read_html(tbl)[0]
course_id_list = []

In [ ]:
soup = BeautifulSoup(tbl,'html.parser')
for link in soup.findAll('a'):
    course_id = link.get('href')
    course_id = course_id.split("=",1)[1]
    course_id_list.append(course_id)
course_id_df = pd.DataFrame(course_id_list)
course_id_df = course_id_df.rename(columns={0: "Course ID"})
course_details['Course ID'] = course_id_df
print(course_details.tail())
all_courses = all_courses.append(course_details).reset_index(drop=True)
print(len(all_courses['Club Name']))
i=i+1

In [ ]:
all_courses.to_csv('course_id_directory.csv',index=False)